In [ ]:
print("Memulai proses setup...")

# Pindah ke direktori konten utama
%cd /content

# Clone repository ComfyUI dan custom node
print("Cloning repository ComfyUI dan custom nodes...")
!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI -q
!git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF -q

# Pindah ke direktori ComfyUI
%cd /content/ComfyUI

print("Menginstal dependensi yang diperlukan...")
!pip install -r requirements.txt accelerate torchsde gradio gguf -q
print("Instalasi dependensi selesai.")

# Instal aria2 untuk mempercepat unduhan
!apt install aria2 -qqy

# Download model-model yang diperlukan menggunakan aria2
print("Mengunduh model Unet...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/QuantStack/Wan2.2-T2V-A14B-GGUF/resolve/main/LowNoise/Wan2.2-T2V-A14B-LowNoise-Q3_K_S.gguf -d /content/ComfyUI/models/unet -o Wan2.2-T2V-A14B-LowNoise-Q3_K_S.gguf

print("Mengunduh model CLIP Encoder...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/city96/umt5-xxl-encoder-gguf/resolve/main/umt5-xxl-encoder-Q3_K_S.gguf -d /content/ComfyUI/models/clip -o umt5-xxl-encoder-Q3_K_S.gguf

print("Mengunduh model VAE...")
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors -d /content/ComfyUI/models/vae -o wan_2.1_vae.safetensors

print("Mengunduh model LoRA...")
!mkdir -p /content/ComfyUI/models/loras/FusionX
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vrgamedevgirl84/Wan14BT2VFusioniX/resolve/main/FusionX_LoRa/Wan2.1_T2V_14B_FusionX_LoRA.safetensors -d /content/ComfyUI/models/loras/FusionX -o Wan2.1_T2V_14B_FusionX_LoRA.safetensors

print("Setup selesai!")

In [ ]:
print("\nMemulai inisialisasi Python...")

# --- Instalasi library yang dibutuhkan ---
try:
    import nest_asyncio
except ImportError:
    print("Menginstal nest_asyncio...")
    !pip install -q nest_asyncio
    import nest_asyncio

try:
    from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
    from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes, CallbackQueryHandler
    from telegram.error import BadRequest, NetworkError, TimedOut
except ImportError:
    print("Menginstal python-telegram-bot...")
    !pip install -q python-telegram-bot
    from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
    from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes, CallbackQueryHandler
    from telegram.error import BadRequest, NetworkError, TimedOut

import sys
import torch
import random
import time
from PIL import Image
import numpy as np
import os
import asyncio
import io
import json
import traceback

nest_asyncio.apply()

# --- Navigasi Direktori ---
try:
    os.chdir('/content/ComfyUI')
    print(f"Berpindah ke direktori: {os.getcwd()}")
except FileNotFoundError:
    print(f"Sudah berada di direktori {os.getcwd()} atau direktori ComfyUI tidak ditemukan.")

# --- Pemuatan Asynchronous Custom Nodes ---
async def load_custom_nodes():
    """Memuat semua custom node yang dibutuhkan secara asynchronous."""
    print("Memulai pemuatan custom nodes...")
    from nodes import load_custom_node
    custom_node_path = os.path.join(os.getcwd(), "custom_nodes", "ComfyUI-GGUF")
    if not os.path.isdir(custom_node_path):
        print(f"!!! WARNING: Direktori custom node tidak ditemukan di {custom_node_path}")
        return
    await load_custom_node(custom_node_path)
    print("Custom nodes berhasil dimuat.")

print("Menjalankan pemuatan custom node...")
asyncio.run(load_custom_nodes())

# --- Import dan Inisialisasi Model Utama ---
from nodes import NODE_CLASS_MAPPINGS, KSamplerAdvanced, VAEDecode, VAEEncode, CLIPTextEncode, EmptyLatentImage, VAELoader, LoraLoaderModelOnly, ImageScale
import comfy.samplers
from comfy_extras.nodes_model_advanced import ModelSamplingSD3

print("Menginisialisasi class-class node...")
try:
    UnetLoaderGGUF = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
    CLIPLoaderGGUF = NODE_CLASS_MAPPINGS["CLIPLoaderGGUF"]()
except KeyError as e:
    print(f"FATAL ERROR: Gagal menemukan node '{e.args[0]}'.")
    print("Pastikan custom node 'ComfyUI-GGUF' terinstal dengan benar.")
    sys.exit(1)

vae_loader = VAELoader()
lora_loader = LoraLoaderModelOnly()

print("\nMemuat model utama (Unet, CLIP, VAE)...")
try:
    unet_model = UnetLoaderGGUF.load_unet("Wan2.2-T2V-A14B-LowNoise-Q3_K_S.gguf")[0]
    clip_model = CLIPLoaderGGUF.load_clip("umt5-xxl-encoder-Q3_K_S.gguf", "wan")[0]
    vae_model = vae_loader.load_vae("wan_2.1_vae.safetensors")[0]

    print("Memuat model LoRA...")
    lora_model = lora_loader.load_lora_model_only(unet_model, "FusionX/Wan2.1_T2V_14B_FusionX_LoRA.safetensors", 1.0)[0]
    print("Semua model berhasil dimuat!")
except Exception as e:
    print(f"!!! FATAL ERROR SAAT MEMUAT MODEL: {e} !!!")
    traceback.print_exc()
    sys.exit(1)

# --- Info Data ---
SAMPLER_INFO = {
    "euler": {"name": "Euler", "speed": "⚡⚡⚡⚡", "quality": "Basic", "best_for": "Quick concepts, testing", "steps": "20-30"},
    "euler_cfg_pp": {"name": "Euler CFG++", "speed": "⚡⚡⚡", "quality": "Good", "best_for": "General purpose", "steps": "25-35"},
    "euler_ancestral": {"name": "Euler Ancestral", "speed": "⚡⚡⚡", "quality": "Creative", "best_for": "Artistic, abstract", "steps": "20-30"},
    "dpmpp_2m": {"name": "DPM++ 2M", "speed": "⚡", "quality": "⭐⭐⭐⭐⭐", "best_for": "Photorealism, portraits", "steps": "20-30"},
    "dpmpp_2m_sde": {"name": "DPM++ 2M SDE", "speed": "⚡", "quality": "⭐⭐⭐⭐⭐", "best_for": "Artistic with details", "steps": "20-30"},
    "dpmpp_2s_ancestral": {"name": "DPM++ 2S Ancestral", "speed": "⚡", "quality": "⭐⭐⭐⭐", "best_for": "Sharp details", "steps": "25-35"},
    "heun": {"name": "Heun", "speed": "⚡⚡", "quality": "⭐⭐⭐⭐", "best_for": "Precise details", "steps": "15-25"},
    "heunpp2": {"name": "Heun++2", "speed": "⚡⚡", "quality": "⭐⭐⭐⭐", "best_for": "High accuracy", "steps": "15-25"},
    "lcm": {"name": "LCM", "speed": "⚡⚡⚡⚡⚡", "quality": "⭐⭐⭐", "best_for": "Real-time, preview", "steps": "4-8"},
    "dpm_fast": {"name": "DPM Fast", "speed": "⚡⚡⚡⚡", "quality": "⭐⭐", "best_for": "Quick testing", "steps": "10-15"},
    "ddim": {"name": "DDIM", "speed": "⚡⚡⚡", "quality": "⭐⭐⭐⭐", "best_for": "Consistent results", "steps": "20-50"},
    "uni_pc": {"name": "UniPC", "speed": "⚡⚡", "quality": "⭐⭐⭐⭐⭐", "best_for": "Scientific accuracy", "steps": "10-25"},
}

SCHEDULER_INFO = {
    "karras": {"name": "Karras", "popularity": "⭐⭐⭐⭐⭐", "best_for": "General purpose, high quality"},
    "normal": {"name": "Normal", "popularity": "⭐⭐⭐⭐", "best_for": "Standard, predictable"},
    "exponential": {"name": "Exponential", "popularity": "⭐⭐⭐", "best_for": "Quick denoising"},
    "simple": {"name": "Simple", "popularity": "⭐⭐", "best_for": "Testing, simplicity"},
    "ddim_uniform": {"name": "DDIM Uniform", "popularity": "⭐⭐⭐", "best_for": "DDIM sampler"},
}

# --- Data untuk pilihan ---
SAMPLERS = comfy.samplers.KSampler.SAMPLERS
SCHEDULERS = comfy.samplers.KSampler.SCHEDULERS
STEPS_OPTIONS = [5, 10, 15, 20]
CFG_OPTIONS = [3, 6, 9, 12, 15]
DENOISE_OPTIONS = [0.5, 0.6, 0.7, 0.8, 0.9]
RESOLUTION_OPTIONS = {
    "512x1024": (512, 1024),
    "1024x512": (1024, 512),
    "1024x1024": (1024, 1024),
    "512x512": (512, 512)
}

# Menyimpan state per user
user_data = {}

# --- Fungsi Generate Gambar T2I ---
def generate_image_t2i(positive_prompt, negative_prompt, width, height, steps, cfg, sampler_name, scheduler):
    """Fungsi untuk menghasilkan gambar dari text (Text to Image)."""
    print("\n--- Memulai Proses Generate T2I ---")
    start_time = time.time()

    try:
        clip_text_encode = CLIPTextEncode()
        empty_latent_image = EmptyLatentImage()
        k_sampler_advanced = KSamplerAdvanced()
        vae_decode = VAEDecode()
        model_sampler_patcher = ModelSamplingSD3()

        with torch.inference_mode():
            print("-> Encoding prompt...")
            positive_conditioning = clip_text_encode.encode(clip_model, positive_prompt)[0]
            negative_conditioning = clip_text_encode.encode(clip_model, negative_prompt)[0]

            print("-> Patching model...")
            model_with_sampler = model_sampler_patcher.patch(lora_model, 1.0)[0]

            print(f"-> Membuat latent {width}x{height}...")
            latent_image = empty_latent_image.generate(width, height, 1)[0]

            seed = random.randint(0, 2**64 - 1)
            print(f"-> Seed: {seed}")

            print(f"-> Sampling dengan {steps} steps...")
            samples = k_sampler_advanced.sample(
                model=model_with_sampler, add_noise="enable", noise_seed=int(seed),
                steps=int(steps), cfg=cfg,
                sampler_name=sampler_name, scheduler=scheduler,
                positive=positive_conditioning, negative=negative_conditioning,
                latent_image=latent_image, start_at_step=0, end_at_step=9999,
                return_with_leftover_noise="disable"
            )[0]

            print("-> Decoding ke gambar...")
            decoded_image_tensor = vae_decode.decode(vae_model, samples)[0]

            image_np = decoded_image_tensor.cpu().numpy()
            image_np_uint8 = (image_np.clip(0, 1) * 255).astype(np.uint8)
            final_image = Image.fromarray(image_np_uint8[0])

            end_time = time.time()
            print(f"--- Selesai ({end_time - start_time:.2f} detik) ---")

            return final_image, seed

    except Exception as e:
        print(f"!!! ERROR: {e} !!!")
        traceback.print_exc()
        return None, None

# --- Fungsi Generate Gambar Img2Img ---
def generate_image_img2img(positive_prompt, negative_prompt, input_image, width, height, steps, cfg, sampler_name, scheduler, denoise):
    """Fungsi untuk menghasilkan gambar dari gambar (Image to Image)."""
    print("\n--- Memulai Proses Generate Img2Img ---")
    start_time = time.time()

    try:
        clip_text_encode = CLIPTextEncode()
        image_scale = ImageScale()
        vae_encode = VAEEncode()
        k_sampler_advanced = KSamplerAdvanced()
        vae_decode = VAEDecode()
        model_sampler_patcher = ModelSamplingSD3()

        with torch.inference_mode():
            print("-> Encoding prompt...")
            positive_conditioning = clip_text_encode.encode(clip_model, positive_prompt)[0]
            negative_conditioning = clip_text_encode.encode(clip_model, negative_prompt)[0]

            print("-> Patching model...")
            model_with_sampler = model_sampler_patcher.patch(lora_model, 1.0)[0]

            print(f"-> Resize gambar ke {width}x{height}...")
            # Convert PIL Image ke tensor format ComfyUI
            img_np = np.array(input_image).astype(np.float32) / 255.0
            img_tensor = torch.from_numpy(img_np)[None,]

            # Resize gambar
            resized_image = image_scale.upscale(img_tensor, "lanczos", width, height, "disabled")[0]

            print("-> Encoding gambar ke latent...")
            latent_image = vae_encode.encode(vae_model, resized_image)[0]

            seed = random.randint(0, 2**64 - 1)
            print(f"-> Seed: {seed}")
            print(f"-> Denoise strength: {denoise}")

            print(f"-> Sampling dengan {steps} steps...")
            samples = k_sampler_advanced.sample(
                model=model_with_sampler, add_noise="enable", noise_seed=int(seed),
                steps=int(steps), cfg=cfg,
                sampler_name=sampler_name, scheduler=scheduler,
                positive=positive_conditioning, negative=negative_conditioning,
                latent_image=latent_image, start_at_step=int((1 - denoise) * steps), end_at_step=9999,
                return_with_leftover_noise="disable"
            )[0]

            print("-> Decoding ke gambar...")
            decoded_image_tensor = vae_decode.decode(vae_model, samples)[0]

            image_np = decoded_image_tensor.cpu().numpy()
            image_np_uint8 = (image_np.clip(0, 1) * 255).astype(np.uint8)
            final_image = Image.fromarray(image_np_uint8[0])

            end_time = time.time()
            print(f"--- Selesai ({end_time - start_time:.2f} detik) ---")

            return final_image, seed

    except Exception as e:
        print(f"!!! ERROR: {e} !!!")
        traceback.print_exc()
        return None, None

# --- Fungsi helper ---
def escape_markdown(text):
    """Escape special characters untuk Markdown di Telegram"""
    escape_chars = ['_', '*', '[', ']', '(', ')', '~', '`', '>', '#', '+', '-', '=', '|', '{', '}', '.', '!']
    for char in escape_chars:
        text = text.replace(char, f'\\{char}')
    return text

# --- Fungsi untuk membuat keyboard ---
def create_main_menu_keyboard():
    """Membuat inline keyboard untuk menu utama"""
    keyboard = [
        [InlineKeyboardButton("🎨 Text to Image", callback_data="menu_t2i")],
        [InlineKeyboardButton("🖼️ Image to Image", callback_data="menu_img2img")],
        [InlineKeyboardButton("📖 Info Sampler", callback_data="info_sampler"),
         InlineKeyboardButton("📅 Info Scheduler", callback_data="info_scheduler")],
        [InlineKeyboardButton("📊 Info Steps & CFG", callback_data="info_steps_cfg"),
         InlineKeyboardButton("💡 Recommendations", callback_data="info_recommendations")]
    ]
    return InlineKeyboardMarkup(keyboard)

def create_sampler_keyboard():
    """Membuat inline keyboard untuk pilihan sampler"""
    keyboard = []
    row = []
    for i, sampler in enumerate(SAMPLERS):
        row.append(InlineKeyboardButton(sampler, callback_data=f"sampler_{sampler}"))
        if (i + 1) % 2 == 0:
            keyboard.append(row)
            row = []
    if row:
        keyboard.append(row)
    keyboard.append([InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")])
    return InlineKeyboardMarkup(keyboard)

def create_scheduler_keyboard():
    """Membuat inline keyboard untuk pilihan scheduler"""
    keyboard = []
    row = []
    for i, scheduler in enumerate(SCHEDULERS):
        row.append(InlineKeyboardButton(scheduler, callback_data=f"scheduler_{scheduler}"))
        if (i + 1) % 2 == 0:
            keyboard.append(row)
            row = []
    if row:
        keyboard.append(row)
    keyboard.append([InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")])
    return InlineKeyboardMarkup(keyboard)

def create_steps_keyboard():
    """Membuat inline keyboard untuk pilihan steps"""
    keyboard = [[InlineKeyboardButton(str(step), callback_data=f"steps_{step}") for step in STEPS_OPTIONS]]
    keyboard.append([InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")])
    return InlineKeyboardMarkup(keyboard)

def create_cfg_keyboard():
    """Membuat inline keyboard untuk pilihan CFG"""
    keyboard = [[InlineKeyboardButton(str(cfg), callback_data=f"cfg_{cfg}") for cfg in CFG_OPTIONS]]
    keyboard.append([InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")])
    return InlineKeyboardMarkup(keyboard)

def create_denoise_keyboard():
    """Membuat inline keyboard untuk pilihan denoise strength"""
    keyboard = [[InlineKeyboardButton(str(d), callback_data=f"denoise_{d}") for d in DENOISE_OPTIONS]]
    keyboard.append([InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")])
    return InlineKeyboardMarkup(keyboard)

def create_resolution_keyboard():
    """Membuat inline keyboard untuk pilihan resolusi"""
    keyboard = []
    for res_name in RESOLUTION_OPTIONS.keys():
        keyboard.append([InlineKeyboardButton(res_name, callback_data=f"resolution_{res_name}")])
    keyboard.append([InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")])
    return InlineKeyboardMarkup(keyboard)

# --- Helper function untuk safe edit message ---
async def safe_edit_message(query, text, reply_markup=None, parse_mode=None):
    """Edit message dengan error handling"""
    try:
        await query.edit_message_text(text=text, reply_markup=reply_markup, parse_mode=parse_mode)
    except BadRequest as e:
        if "Message is not modified" in str(e):
            pass
        elif "can't parse entities" in str(e):
            try:
                await query.edit_message_text(text=text, reply_markup=reply_markup, parse_mode=None)
            except:
                pass
        else:
            raise

async def show_main_menu(update_or_query, is_query=False):
    """Menampilkan menu utama"""
    menu_text = """🎨 ComfyUI GGUF Image Generator Bot

Pilih mode generasi:
• Text to Image - Generate dari prompt text
• Image to Image - Transform gambar existing

Info & Panduan:
• Info Sampler - Pelajari tentang sampler
• Info Scheduler - Pelajari tentang scheduler
• Info Steps & CFG - Pelajari parameter
• Recommendations - Kombinasi terbaik

Selamat berkreasi! ✨
"""

    if is_query:
        await safe_edit_message(update_or_query, menu_text, reply_markup=create_main_menu_keyboard())
    else:
        await update_or_query.message.reply_text(menu_text, reply_markup=create_main_menu_keyboard())

# --- Handler Telegram Bot ---
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handler untuk command /start"""
    await show_main_menu(update, is_query=False)

async def button_callback(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handler untuk callback dari inline keyboard"""
    query = update.callback_query
    await query.answer()

    user_id = update.effective_user.id
    data = query.data

    # Handle menu utama
    if data == "menu_t2i":
        await safe_edit_message(query, "🎨 Mode: Text to Image\n\n📝 Silakan kirim prompt Anda.\n\nContoh: a cinematic photo of a lion")
        user_data[user_id] = {'mode': 't2i', 'waiting_prompt': True}
        return

    elif data == "menu_img2img":
        await safe_edit_message(query, "🖼️ Mode: Image to Image\n\n📸 Silakan kirim gambar terlebih dahulu.")
        user_data[user_id] = {'mode': 'img2img', 'waiting_image': True}
        return

    elif data == "back_to_menu":
        if user_id in user_data:
            del user_data[user_id]
        await show_main_menu(query, is_query=True)
        return

    # Handle info pages
    elif data == "info_sampler":
        info_text = "📖 Info Sampler\n\n"
        for key in ["dpmpp_2m", "dpmpp_2m_sde", "euler_ancestral", "lcm", "heun"]:
            if key in SAMPLER_INFO:
                s = SAMPLER_INFO[key]
                info_text += f"{s['name']}\n"
                info_text += f"├ Speed: {s['speed']}\n"
                info_text += f"├ Quality: {s['quality']}\n"
                info_text += f"├ Best for: {s['best_for']}\n"
                info_text += f"└ Steps: {s['steps']}\n\n"

        keyboard = [[InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")]]
        await safe_edit_message(query, info_text, reply_markup=InlineKeyboardMarkup(keyboard))
        return

    elif data == "info_scheduler":
        info_text = "📅 Info Scheduler\n\n"
        for key, s in SCHEDULER_INFO.items():
            info_text += f"{s['name']}\n"
            info_text += f"├ Popularity: {s['popularity']}\n"
            info_text += f"└ Best for: {s['best_for']}\n\n"

        keyboard = [[InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")]]
        await safe_edit_message(query, info_text, reply_markup=InlineKeyboardMarkup(keyboard))
        return

    elif data == "info_steps_cfg":
        info_text = """📊 Info Steps & CFG

Steps (Denoising Steps)
• 5-10: Cepat, kasar, cocok untuk preview
• 10-15: Balance speed & quality
• 15-20: Good quality, recommended
• 20+: High quality, lambat

➡️ Steps rendah → hasil kasar
➡️ Steps tinggi → detail lebih baik

CFG Scale (Classifier-Free Guidance)
• 1-3: Sangat kreatif, bebas
• 3-7: Balance kreativitas & prompt
• 7-12: Mengikuti prompt ketat
• 12+: Sangat ketat, bisa oversaturated

➡️ CFG rendah → bebas & kreatif
➡️ CFG tinggi → kaku tapi sesuai prompt

Denoise Strength (Img2Img Only)
• 0.3-0.5: Perubahan minimal, pertahankan struktur
• 0.5-0.7: Perubahan sedang, recommended
• 0.7-0.9: Perubahan besar, kreatif
• 0.9-1.0: Hampir seperti gambar baru

➡️ Denoise rendah → mirip gambar asli
➡️ Denoise tinggi → lebih kreatif
"""
        keyboard = [[InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")]]
        await safe_edit_message(query, info_text, reply_markup=InlineKeyboardMarkup(keyboard))
        return

    elif data == "info_recommendations":
        info_text = """💡 Recommended Combinations

🎬 Photorealistic
• Sampler: dpmpp_2m
• Scheduler: karras
• Steps: 20-30 | CFG: 7.5
• Denoise (img2img): 0.6-0.7

🎨 Artistic
• Sampler: dpmpp_2m_sde
• Scheduler: karras
• Steps: 25-35 | CFG: 5-7
• Denoise (img2img): 0.7-0.8

⚡ Fast Concept
• Sampler: lcm
• Scheduler: normal
• Steps: 4-8 | CFG: 2-4
• Denoise (img2img): 0.5-0.6

👤 Portraits
• Sampler: dpmpp_2m
• Scheduler: karras
• Steps: 30-40 | CFG: 7-8
• Denoise (img2img): 0.5-0.6

🏞 Landscapes
• Sampler: euler_ancestral
• Scheduler: normal
• Steps: 25-35 | CFG: 6-7
• Denoise (img2img): 0.6-0.7

🎭 Anime/Art
• Sampler: dpmpp_2s_ancestral
• Scheduler: karras
• Steps: 20-30 | CFG: 5-6
• Denoise (img2img): 0.7-0.8
"""
        keyboard = [[InlineKeyboardButton("🔙 Back to Menu", callback_data="back_to_menu")]]
        await safe_edit_message(query, info_text, reply_markup=InlineKeyboardMarkup(keyboard))
        return

    # Handle generation flow
    if user_id not in user_data or 'prompt' not in user_data[user_id]:
        await safe_edit_message(query, "❌ Session expired. Silakan mulai dari menu.")
        return

    user_state = user_data[user_id]

    try:
        if data.startswith('sampler_'):
            sampler = data.replace('sampler_', '')
            user_state['sampler'] = sampler
            await safe_edit_message(query, f"✅ Sampler: {sampler}\n\n2️⃣ Pilih Scheduler:", reply_markup=create_scheduler_keyboard())

        elif data.startswith('scheduler_'):
            scheduler = data.replace('scheduler_', '')
            user_state['scheduler'] = scheduler
            await safe_edit_message(query, f"✅ Scheduler: {scheduler}\n\n3️⃣ Pilih Steps:", reply_markup=create_steps_keyboard())

        elif data.startswith('steps_'):
            steps = int(data.replace('steps_', ''))
            user_state['steps'] = steps
            await safe_edit_message(query, f"✅ Steps: {steps}\n\n4️⃣ Pilih CFG Scale:", reply_markup=create_cfg_keyboard())

        elif data.startswith('cfg_'):
            cfg = float(data.replace('cfg_', ''))
            user_state['cfg'] = cfg

            # Jika img2img, tampilkan pilihan denoise
            if user_state.get('mode') == 'img2img':
                await safe_edit_message(query, f"✅ CFG: {cfg}\n\n5️⃣ Pilih Denoise Strength:", reply_markup=create_denoise_keyboard())
            else:
                await safe_edit_message(query, f"✅ CFG: {cfg}\n\n5️⃣ Pilih Resolusi:", reply_markup=create_resolution_keyboard())

        elif data.startswith('denoise_'):
            denoise = float(data.replace('denoise_', ''))
            user_state['denoise'] = denoise
            await safe_edit_message(query, f"✅ Denoise: {denoise}\n\n6️⃣ Pilih Resolusi:", reply_markup=create_resolution_keyboard())

        elif data.startswith('resolution_'):
            resolution = data.replace('resolution_', '')
            width, height = RESOLUTION_OPTIONS[resolution]
            user_state['width'] = width
            user_state['height'] = height

            await safe_edit_message(query, f"✅ Resolusi: {resolution}\n\n🎨 Memulai generate...\nMohon tunggu...")

            status_msg = await query.message.reply_text("⏳ Sedang memproses...")

            # Generate berdasarkan mode
            if user_state.get('mode') == 'img2img':
                image, seed = generate_image_img2img(
                    positive_prompt=user_state['prompt'],
                    negative_prompt='blurry, low quality, static, frame, text, watermark',
                    input_image=user_state['input_image'],
                    width=width, height=height, steps=user_state['steps'],
                    cfg=user_state['cfg'], sampler_name=user_state['sampler'],
                    scheduler=user_state['scheduler'],
                    denoise=user_state['denoise']
                )
            else:
                image, seed = generate_image_t2i(
                    positive_prompt=user_state['prompt'],
                    negative_prompt='blurry, low quality, static, frame, text, watermark',
                    width=width, height=height, steps=user_state['steps'],
                    cfg=user_state['cfg'], sampler_name=user_state['sampler'],
                    scheduler=user_state['scheduler']
                )

            try:
                await status_msg.delete()
            except:
                pass

            if image:
                bio = io.BytesIO()
                image.save(bio, format='PNG')
                bio.seek(0)

                mode_text = "Image to Image" if user_state.get('mode') == 'img2img' else "Text to Image"
                caption = f"""✅ Generated Image ({mode_text})

📝 Prompt: {user_state['prompt']}

⚙️ Parameters:
• Sampler: {user_state['sampler']}
• Scheduler: {user_state['scheduler']}
• Steps: {user_state['steps']}
• CFG: {user_state['cfg']}"""

                if user_state.get('mode') == 'img2img':
                    caption += f"\n• Denoise: {user_state['denoise']}"

                caption += f"""
• Resolution: {width}x{height}
• Seed: {seed}"""

                try:
                    await query.message.reply_photo(photo=bio, caption=caption, read_timeout=60, write_timeout=60)
                except:
                    bio.seek(0)
                    await query.message.reply_document(document=bio, filename=f"generated_{seed}.png", caption=caption)

                # Kembali ke menu utama
                await query.message.reply_text("Pilih opsi berikutnya:", reply_markup=create_main_menu_keyboard())
            else:
                await query.message.reply_text("❌ Error saat generate. Coba lagi.")

            del user_data[user_id]

    except Exception as e:
        print(f"Error: {e}")
        traceback.print_exc()
        try:
            await query.message.reply_text(f"❌ Error: {str(e)}")
        except:
            pass

async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handler untuk menerima text prompt"""
    user_id = update.effective_user.id

    if user_id in user_data:
        state = user_data[user_id]

        # Handle prompt untuk mode t2i
        if state.get('waiting_prompt'):
            prompt = update.message.text.strip()
            user_data[user_id] = {
                'mode': state.get('mode'),
                'prompt': prompt,
                'input_image': state.get('input_image')
            }

            await update.message.reply_text(
                f"📝 Prompt: {prompt}\n\n1️⃣ Pilih Sampler:",
                reply_markup=create_sampler_keyboard()
            )

        # Handle prompt setelah upload gambar untuk img2img
        elif state.get('waiting_prompt_after_image'):
            prompt = update.message.text.strip()
            user_data[user_id]['prompt'] = prompt
            user_data[user_id]['waiting_prompt_after_image'] = False

            await update.message.reply_text(
                f"📝 Prompt: {prompt}\n\n1️⃣ Pilih Sampler:",
                reply_markup=create_sampler_keyboard()
            )
        else:
            await update.message.reply_text("Gunakan /start untuk memulai")
    else:
        await update.message.reply_text("Gunakan /start untuk memulai")

async def handle_photo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handler untuk menerima foto"""
    user_id = update.effective_user.id

    if user_id in user_data and user_data[user_id].get('waiting_image'):
        try:
            # Download foto
            photo_file = await update.message.photo[-1].get_file()
            photo_bytes = await photo_file.download_as_bytearray()

            # Convert ke PIL Image
            input_image = Image.open(io.BytesIO(photo_bytes)).convert('RGB')

            user_data[user_id]['input_image'] = input_image
            user_data[user_id]['waiting_image'] = False
            user_data[user_id]['waiting_prompt_after_image'] = True

            await update.message.reply_text(
                "✅ Gambar diterima!\n\n📝 Sekarang kirim prompt untuk transformasi gambar.\n\nContoh: transform into anime style"
            )
        except Exception as e:
            print(f"Error handling photo: {e}")
            traceback.print_exc()
            await update.message.reply_text("❌ Error saat memproses gambar. Coba lagi.")
    else:
        await update.message.reply_text("Gunakan /start dan pilih Image to Image terlebih dahulu")

async def error_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handler untuk error"""
    print(f"Error: {context.error}")
    traceback.print_exc()

# --- Main Bot ---
async def main():
    """Fungsi utama untuk menjalankan bot"""

    BOT_TOKEN = "7729445464:AAE01xKyGN9DDxY92lqSlFTAJwytmXLoBc8"

    if BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
        print("\n!!! ERROR: Ganti BOT_TOKEN !!!")
        return

    print("\n🤖 Memulai Telegram Bot...")

    application = (
        Application.builder()
        .token(BOT_TOKEN)
        .read_timeout(60)
        .write_timeout(60)
        .connect_timeout(60)
        .pool_timeout(60)
        .build()
    )

    application.add_error_handler(error_handler)
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CallbackQueryHandler(button_callback))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    print("✅ Bot siap! Fitur T2I dan Img2Img aktif!")
    await application.run_polling(allowed_updates=Update.ALL_TYPES, drop_pending_updates=True)

if __name__ == '__main__':
    asyncio.run(main())